<a href="https://codeimmersives.com"><img src = "https://www.codeimmersives.com/wp-content/uploads/2019/09/CodeImmersives_Logo_RGB_NYC_BW.png" width = 400> </a>


<h1 align=center><font size = 5>Agenda</font></h1>

### Agenda
<div class="alert alert-block alert-info" style="margin-top: 20px">

1.  [Review](#0)<br>
2.  [memory]
3.  [SELECT DISTINCT](#2)<br>
4.  [COUNT](#4) <br>
5.  [ALTER TABLE](#6) <br>     
6.  [MIN and CAST](#8) <br> 
7.  [MAX](#10) <br>
8.  [AVG](#12) <br>
9.  [SUM](#12) <br>
10. [Exercise 1](#14) <br>    
11. [Exercise 2](#16) <br>
12. [Exercise 3](#18) <br>
    
</div>
<hr>

In [ ]:
con = sqlite3.connect(":memory:")

<h4>Back up the database</h4>

In [6]:
import sqlite3

def progress(status, remaining, total):
    print(f'Copied {total-remaining} of {total} pages...')

conn = sqlite3.connect('titanic.db')
bck = sqlite3.connect('backup.db')
with bck:
    conn.backup(bck, pages=1, progress=progress)
bck.close()
conn.close()

Copied 1 of 15 pages...
Copied 2 of 15 pages...
Copied 3 of 15 pages...
Copied 4 of 15 pages...
Copied 5 of 15 pages...
Copied 6 of 15 pages...
Copied 7 of 15 pages...
Copied 8 of 15 pages...
Copied 9 of 15 pages...
Copied 10 of 15 pages...
Copied 11 of 15 pages...
Copied 12 of 15 pages...
Copied 13 of 15 pages...
Copied 14 of 15 pages...
Copied 15 of 15 pages...


<h4>Copy a database to memory</h4>

In [83]:
import sqlite3

source = sqlite3.connect('existing_db.db')
dest = sqlite3.connect(':memory:')
source.backup(dest)

<h4>Use executescript(sql) to run several scripts at once</h4> 

In [7]:
import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.executescript('''
    create table person(
        firstname,
        lastname,
        age
    );

    create table book(
        title,
        author,
        published
    );

    insert into book(title, author, published)
    values (
        'Dirk Gently''s Holistic Detective Agency',
        'Douglas Adams',
        1987
    );
 ''')
conn.commit()
c.close()   # <=== cursor is closed
# conn.row_factory = sqlite3.Row
c = conn.cursor()
c.execute('select * from book')
r = c.fetchone()
print(r)
for member in r:
    print(member)

("Dirk Gently's Holistic Detective Agency", 'Douglas Adams', 1987)
Dirk Gently's Holistic Detective Agency
Douglas Adams
1987


<h4>Exercise<br>
Create a database called Library.db<br>
Create a table named 'Books' it has 3 fields (id TEXT, Title TEXT and Author TEXT)<br>
id, Title, Author<br>
001, Outliers, Malcolm Gladwell<br>
002, Humble Pie, Matt Parker<br>
003, 52 Small Changes, Brett Blumenthal<br>
004, Blink, Malcolm Gladwell<br>
005, The Tipping Point, Malcolm Gladwell<br>
006, All the Powers of Earth, Sidney Blumenthal


</h4>

In [85]:
import sqlite3
conn = sqlite3.connect("Library.db")
c = conn.cursor()

c.execute("""CREATE TABLE IF NOT EXISTS books(
    id TEXT,
    Title TEXT,
    Author TEXT,
    Primary Key(id))
""")

all_books = [
    ("001", "Outliers", "Malcolm Gladwell"),
    ("002", "Humble Pie", "Matt Parker"),
    ("003", "52 Small Changes", "Brett Blumenthal"),
    ("004", "Blink", "Malcolm Gladwell"),
    ("005", "The Tipping Point", "Malcolm Gladwell"),
    ("006", "All the Powers of Earth", "Sidney Blumenthal"),
]
c.executemany("""INSERT INTO books VALUES (?,?,?)""", all_books)
conn.commit()
conn.close()





IntegrityError: UNIQUE constraint failed: books.id

In [86]:
conn = sqlite3.connect('Library.db')
c = conn.cursor()

c.execute('SELECT * FROM books')
data = c.fetchall()
print(data)

conn.commit()
conn.close()


[('001', 'Outliers', 'Malcolm Gladwell'), ('002', 'Humble Pie', 'Matt Parker'), ('003', '52 Small Changes', 'Brett Blumenthal'), ('004', 'Blink', 'Malcolm Gladwell'), ('005', 'The Tipping Point', 'Malcolm Gladwell'), ('006', 'All the Powers of Earth', 'Sidney Blumenthal')]


<h4>Every time you run the code the inserts are repeated<br>
To get a unduplicated list of the book use:<br>
SELECT DISTINCT id,title,author FROM books

In [87]:
conn = sqlite3.connect('Library.db')
c = conn.cursor()
c.execute("SELECT DISTINCT id, title, author FROM books")
books = c.fetchall()

for i in books:
    print(i)
    print(30*"=")

conn.close()


('001', 'Outliers', 'Malcolm Gladwell')
('002', 'Humble Pie', 'Matt Parker')
('003', '52 Small Changes', 'Brett Blumenthal')
('004', 'Blink', 'Malcolm Gladwell')
('005', 'The Tipping Point', 'Malcolm Gladwell')
('006', 'All the Powers of Earth', 'Sidney Blumenthal')


<h4>Use the SELECT count() to add a total count of all items in the table with or without a qualifier

In [14]:
import sqlite3

conn = sqlite3.connect(r"library.db")
c = conn.cursor()
c.execute('SELECT count() FROM books')  # Counts all values in the table based on the expression
totalBooks = c.fetchone()
print(totalBooks)
#
# for book in totalBooks:
#     print(book)
#     print('='*30)
conn.close()

(6,)


In [60]:
import sqlite3

conn = sqlite3.connect(r"library.db")
c = conn.cursor()
c.execute('SELECT count(DISTINCT id) FROM books')  # Counts all values in the table based on the expression
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
conn.close()

(6,)


<h4>ALTER TABLE</H4>

<H4>ALTER TABLE to rename a table</H4>

In [16]:
import sqlite3

conn = sqlite3.connect(r"library.db")
c = conn.cursor()

# c.execute('ALTER TABLE my_books RENAME TO books')  
# conn.commit()

c.execute('ALTER TABLE books RENAME TO my_books')  
conn.commit()
c.execute('SELECT * FROM my_books')  
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)

# Rename it back to books
c.execute('ALTER TABLE my_books RENAME TO books')  
conn.commit()
conn.close()


('001', 'Outliers', 'Malcolm Gladwell')
('002', 'Humble Pie', 'Matt Parker')
('003', '52 Small Changes', 'Brett Blumenthal')
('004', 'Blink', 'Malcolm Gladwell')
('005', 'The Tipping Point', 'Malcolm Gladwell')
('006', 'All the Powers of Earth', 'Sidney Blumenthal')


<H4>ALTER TABLE RENAME|ADD COLUMN</H4>
NOTE: DROP COLUMN is not supported in SQLite!!!

In [ ]:
import sqlite3

conn = sqlite3.connect(r"library.db")
c = conn.cursor()
c.execute('ALTER TABLE books RENAME COLUMN title TO sub_title')
conn.commit()
c.execute('SELECT * FROM books')
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
    
# Rename it back
c.execute('ALTER TABLE books RENAME COLUMN sub_title TO title')  
conn.commit()
c.execute('SELECT * FROM books')
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
conn.close()

In [17]:
import sqlite3

conn = sqlite3.connect(r"library.db")
c = conn.cursor()
c.execute('ALTER TABLE books ADD COLUMN price TEXT') # This can be REAL type as well
conn.commit()
c.execute('ALTER TABLE books ADD COLUMN ebook_price TEXT') # This can be REAL type as well
conn.commit()
c.execute('SELECT * FROM books')
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
    
conn.close()

('001', 'Outliers', 'Malcolm Gladwell', None, None)
('002', 'Humble Pie', 'Matt Parker', None, None)
('003', '52 Small Changes', 'Brett Blumenthal', None, None)
('004', 'Blink', 'Malcolm Gladwell', None, None)
('005', 'The Tipping Point', 'Malcolm Gladwell', None, None)
('006', 'All the Powers of Earth', 'Sidney Blumenthal', None, None)


<H4>Exercise</H4>
Set price of all Gladwell books at 12.99<br>
Set the price of all Blumnethal books at 9.99<br>
Set the price of Matt Parker's book at 7.99<br>

In [21]:
import sqlite3

conn = sqlite3.connect(r"library.db")
c = conn.cursor()
c.execute('''
UPDATE  books SET
price = 12.99
WHERE author = 'Malcolm Gladwell'
''')
c.execute('''
UPDATE  books SET
price = 9.99
WHERE author LIKE '%Blumenthal'
''')
c.execute('''
UPDATE  books SET
price = 7.99
WHERE author = 'Matt Parker'
''')
conn.commit()
c.execute('SELECT * FROM books')
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
    
conn.close()


('001', 'Outliers', 'Malcolm Gladwell', '12.99', None)
('002', 'Humble Pie', 'Matt Parker', '7.99', None)
('003', '52 Small Changes', 'Brett Blumenthal', '9.99', None)
('004', 'Blink', 'Malcolm Gladwell', '12.99', None)
('005', 'The Tipping Point', 'Malcolm Gladwell', '12.99', None)
('006', 'All the Powers of Earth', 'Sidney Blumenthal', '9.99', None)


<h4>MIN, MAX, CAST</H4>

In [22]:
import sqlite3

conn = sqlite3.connect(r"library.db")
c = conn.cursor()
c.execute('SELECT MIN(CAST(price as REAL)) FROM books')
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
c.execute('SELECT MAX(CAST(price as REAL)) FROM books')
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
conn.close()

(7.99,)
(12.99,)


<h4>SUM, AVG</H4>

In [24]:
import sqlite3

conn = sqlite3.connect(r"library.db")
c = conn.cursor()
c.execute('SELECT SUM(CAST(price as REAL)) FROM books')
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
c.execute('SELECT AVG(CAST(price as REAL)) FROM books')
books = c.fetchall()
#
for book in books:
    print(book)
    print('='*30)
conn.close()

(66.94,)
(11.156666666666666,)


This notebook is part of a course at www.codeimmersives.com called SQL-NOSQL. If you accessed this notebook outside the course, you can get more information about this course online by clicking here.

<hr>

Copyright &copy; 2021  Code Immersives